In [172]:
def warn(*args, **kwargs): pass
import warnings
from random import randint
import random
warnings.warn = warn

In [191]:
cat = [
    ['економіка','гроші'],
    ['спорт','волейбол'],
    ['ідеологія','політика'],
    ['кімната','дім'],
    ['освіта','предмети','математика'],
    ['мистецтво','творчість'],
    ['тварини','самці'],
    ['їжа','борщ'],
    ['озеро','гора']]
cat_name = ['Економіка   -- 1','Спорт       -- 2','Політика    -- 3','Дім         -- 4',
            'Освіта      -- 5','Мистецтво   -- 6','Тваринки    -- 7','Їжа         -- 8',
            'Природа     -- 9']
cat_name_p = ['Економіка','Спорт','Політика','Дім',
            'Освіта','Мистецтво','Тваринки','Їжа',
            'Природа']

In [192]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('word_vec', binary=False)

In [197]:
def generate(cat_n):
    a = list(model.most_similar(positive=cat[cat_n],topn=18))
    b =[]
    for h in a:
        g = h[0]
        if (g[0].isupper()==True):continue
        b.append(g)
    ind = random.sample(range(0, len(b)), 4)
    res = [b[i] for i in ind]
    #########################################################  
    i =0
    while(i==0):
        rand_cat = randint(0, 8)
        if(rand_cat!=cat_n): i=1
    a = list(model.most_similar(positive=cat[rand_cat],topn=18))
    b =[]
    for h in a:
        g = h[0]
        if (g[0].isupper()==True):continue
        b.append(g)
    rand_n = randint(0, (len(b)-1))
    rand_pos = randint(0, (len(res)-1))
    res.insert(rand_pos, b[rand_n]) 
    return res, rand_pos, rand_cat

In [198]:
# Echo server program
import socket
import numpy as np
import time
HOST = ''                 # Symbolic name meaning all available interfaces
PORT = 50008              # Arbitrary non-privileged port
i=0
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(1)
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        print('Game started')
        ############################ Choosing game mode ##############################
        iter1 = 'У грі доступні такі категорії:'+'\n'+'\n'.join(cat_name)+'\n'+ "Виберіть категорію для початку гри!"
        conn.sendall(str(iter1).encode("utf-8"))
        cat_number = conn.recv(1024)
        cat_number = int(cat_number.decode("utf-8")) - 1
        if (cat_number not in set(range(0,len(cat_name)))): iter2 = '404' #checking dump user
        else:
            print("Користувачем обрана категорія   "+cat_name_p[cat_number])
            iter2 = 'Вами вибрана категорія \''+ cat_name_p[cat_number]+'\''+'\n'+'У вас для початку є три права на помилку'
        conn.sendall(str(iter2).encode("utf-8"))
        ### Init game and start
        life = 3
        points = 0
        ############################ Step by step iterations #########################
        while life>0:
            itern = 'Ваш рахунок: '+str(points)+'\n'+ 'У вас ще помилок: '+ str(life)+'\n'
            game = generate(cat_number)
            itern = '\n' + itern + ('   '.join(game[0])) + '\n' +("Який номер слова з списку, найменше підходить під обрану категорію? ")
            conn.sendall(str(itern).encode("utf-8"))
            ans = conn.recv(1024)
            ans = int(ans.decode("utf-8"))
            if(int(ans)==game[1]+1):
                points+=1
                iternn = 'Вітаю, Ваша відповідь правильна!'
            else:
                iternn = 'На жаль цього разу Ви не вгадали'+ '\n'+ 'Правильна відповідь : \''+ game[0][game[1]]+'\' що відноситься до категорії '+ cat_name_p[game[2]]
                if (ans not in set(range(1,6))): iternn = iternn +' \n'+'-1 життя... наступного разу вводь число від 1 до 5'
                life = life-1
            conn.sendall(str(iternn).encode("utf-8"))
            if not life: 
                break
                print('Sorry. Connection lost') 
        print('Game is over')
        dd = 'end'
        conn.sendall(dd.encode("utf-8"))
        time.sleep(1.5)
        finals = 'Гра закінчилася з рахунком:  '+ str(points) + '\n'+'Дякую, Ви молодець!'
        conn.sendall(finals.encode("utf-8"))
conn.close()
s.close()              

Connected by ('127.0.0.1', 55252)
Game started
Користувачем обрана категорія   Природа
Game is over
